## SQL at Scale with Spark SQL

Welcome to the SQL mini project. For this project, you will use the Databricks Platform and work through a series of exercises using Spark SQL. The dataset size may not be too big but the intent here is to familiarize yourself with the Spark SQL interface which scales easily to huge datasets, without you having to worry about changing your SQL queries. 

The data you need is present in the mini-project folder in the form of three CSV files. This data will be imported in Databricks to create the following tables under the __`country_club`__ database.

<br>
1. The __`bookings`__ table,
2. The __`facilities`__ table, and
3. The __`members`__ table.

You will be uploading these datasets shortly into the Databricks platform to understand how to create a database within minutes! Once the database and the tables are populated, you will be focusing on the mini-project questions.

In the mini project, you'll be asked a series of questions. You can solve them using the databricks platform, but for the final deliverable,
please download this notebook as an IPython notebook (__`File -> Export -> IPython Notebook`__) and upload it to your GitHub.

### Creating the Database

We will first create our database in which we will be creating our three tables of interest

In [0]:
%sql 
drop database if exists country_club cascade;
create database country_club;
show databases;

### Creating the Tables

In this section, we will be creating the three tables of interest and populate them with the data from the CSV files already available to you. 
To get started, first upload the three CSV files to the DBFS as depicted in the following figure

![](https://i.imgur.com/QcCruBr.png)


Once you have done this, please remember to execute the following code to build the dataframes which will be saved as tables in our database

In [0]:
# File location and type
file_location_bookings = "/FileStore/tables/Bookings.csv"
file_location_facilities = "/FileStore/tables/Facilities.csv"
file_location_members = "/FileStore/tables/Members.csv"

file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
bookings_df = (spark.read.format(file_type) 
                    .option("inferSchema", infer_schema) 
                    .option("header", first_row_is_header) 
                    .option("sep", delimiter) 
                    .load(file_location_bookings))

facilities_df = (spark.read.format(file_type) 
                      .option("inferSchema", infer_schema) 
                      .option("header", first_row_is_header) 
                      .option("sep", delimiter) 
                      .load(file_location_facilities))

members_df = (spark.read.format(file_type) 
                      .option("inferSchema", infer_schema) 
                      .option("header", first_row_is_header) 
                      .option("sep", delimiter) 
                      .load(file_location_members))

### Viewing the dataframe schemas

We can take a look at the schemas of our potential tables to be written to our database soon

In [0]:
print('Bookings Schema')
print('number rows =', bookings_df.count())
bookings_df.printSchema()

print('\nFacilities Schema')
print('number rows =', facilities_df.count())
facilities_df.printSchema()

print('\nMembers Schema')
print('number rows =', members_df.count())
members_df.printSchema()

### Create permanent tables
We will be creating three permanent tables here in our __`country_club`__ database as we discussed previously with the following code

In [0]:
permanent_table_name_bookings = "country_club.Bookings"
bookings_df.write.format("parquet").saveAsTable(permanent_table_name_bookings)

permanent_table_name_facilities = "country_club.Facilities"
facilities_df.write.format("parquet").saveAsTable(permanent_table_name_facilities)

permanent_table_name_members = "country_club.Members"
members_df.write.format("parquet").saveAsTable(permanent_table_name_members)

# START AGAIN HERE

### Refresh tables and check them

In [0]:
%sql
use country_club;
REFRESH table bookings;
REFRESH table facilities;
REFRESH table members;
show tables;

database,tableName,isTemporary
country_club,bookings,false
country_club,facilities,false
country_club,members,false


### Test a sample SQL query

__Note:__ You can use __`%sql`__ at the beginning of a cell and write SQL queries directly as seen in the following cell. Neat isn't it!

In [0]:
%sql
select * from bookings limit 3

bookid,facid,memid,starttime,slots
0,3,1,2012-07-03 11:00:00,2
1,4,1,2012-07-03 08:00:00,2
2,6,0,2012-07-03 18:00:00,2


#### Q1: Some of the facilities charge a fee to members, but some do not. Please list the names of the facilities that do.

###### Q1: ANSWER

In [0]:
%sql
select name
  from facilities
  where membercost > 0

name
Tennis Court 1
Tennis Court 2
Massage Room 1
Massage Room 2
Squash Court


####  Q2: How many facilities do not charge a fee to members?

###### get facilities that do not charge fee to members

In [0]:
%sql
select name
  from facilities
  where membercost = 0

name
Badminton Court
Table Tennis
Snooker Table
Pool Table


###### Q2: ANSWER

In [0]:
%sql
select count(name) as member_free_facilities
  from facilities
  where membercost = 0

member_free_facilities
4


#### Q3: How can you produce a list of facilities that charge a fee to members, where the fee is less than 20% of the facility's monthly maintenance cost? 
#### Return the facid, facility name, member cost, and monthly maintenance of the facilities in question.

###### get all facilities meeting criteria

In [0]:
%sql
select *
  from facilities
  where membercost < 0.20 * monthlymaintenance

facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,Tennis Court 1,5.0,25.0,10000,200
1,Tennis Court 2,5.0,25.0,8000,200
2,Badminton Court,0.0,15.5,4000,50
3,Table Tennis,0.0,5.0,320,10
4,Massage Room 1,9.9,80.0,4000,3000
5,Massage Room 2,9.9,80.0,4000,3000
6,Squash Court,3.5,17.5,5000,80
7,Snooker Table,0.0,5.0,450,15
8,Pool Table,0.0,5.0,400,15


###### Q3: ANSWER

In [0]:
%sql
select facid, name, membercost, monthlymaintenance
  from facilities
  where membercost > 0 and membercost < 0.20 * monthlymaintenance

facid,name,membercost,monthlymaintenance
0,Tennis Court 1,5.0,200
1,Tennis Court 2,5.0,200
4,Massage Room 1,9.9,3000
5,Massage Room 2,9.9,3000
6,Squash Court,3.5,80


#### Q4: How can you retrieve the details of facilities with ID 1 and 5? Write the query without using the OR operator.

###### Q4: ANSWER

In [0]:
%sql
select *
  from facilities
  where facid in (1,5)

facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
1,Tennis Court 2,5.0,25.0,8000,200
5,Massage Room 2,9.9,80.0,4000,3000


#### Q5: How can you produce a list of facilities, with each labelled as 'cheap' or 'expensive', depending on if their monthly maintenance cost is more than $100? 
#### Return the name and monthly maintenance of the facilities in question.

###### Q5: ANSWER

In [0]:
%sql
select name, monthlymaintenance,
       case when monthlymaintenance > 100 then 'expensive'
       else 'cheap' end as cost_label
  from facilities

name,monthlymaintenance,cost_label
Tennis Court 1,200,expensive
Tennis Court 2,200,expensive
Badminton Court,50,cheap
Table Tennis,10,cheap
Massage Room 1,3000,expensive
Massage Room 2,3000,expensive
Squash Court,80,cheap
Snooker Table,15,cheap
Pool Table,15,cheap


#### Q6: You'd like to get the first and last name of the last member(s) who signed up. Do not use the LIMIT clause for your solution.

###### get list of members and their sign-up date

In [0]:
%sql
select surname, firstname, joindate
  from members

surname,firstname,joindate
GUEST,GUEST,2012-07-01 00:00:00
Smith,Darren,2012-07-02 12:02:05
Smith,Tracy,2012-07-02 12:08:23
Rownam,Tim,2012-07-03 09:32:15
Joplette,Janice,2012-07-03 10:25:05
Butters,Gerald,2012-07-09 10:44:09
Tracy,Burton,2012-07-15 08:52:55
Dare,Nancy,2012-07-25 08:59:12
Boothe,Tim,2012-07-25 16:02:35
Stibbons,Ponder,2012-07-25 17:09:05


###### find most recent sign-up date

In [0]:
%sql
select max(joindate) as max_date
  from members

max_date
2012-09-26 18:08:45


###### this approach to answer give performance warning

In [0]:
%sql
select surname, firstname, joindate
  from members
  where joindate = (select max(joindate) from members)

surname,firstname,joindate
Smith,Darren,2012-09-26 18:08:45


###### Q6: ANSWER (no performance warning)

In [0]:
%sql
select surname, firstname, joindate
from members
where joindate in (select max(joindate) from members)

surname,firstname,joindate
Smith,Darren,2012-09-26 18:08:45


####  Q7: How can you produce a list of all members who have used a tennis court?
- Include in your output the name of the court, and the name of the member formatted as a single column. 
- Ensure no duplicate data
- Also order by the member name.

###### get list of tennis court facilities

In [0]:
%sql
select *
  from facilities
  where name like '%Tennis Court%'

facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,Tennis Court 1,5.0,25.0,10000,200
1,Tennis Court 2,5.0,25.0,8000,200


###### add column for full member name

In [0]:
%sql
select memid, zipcode, surname, firstname,
       (memid + zipcode) as temp_value,
       concat_ws(' ', firstname,surname) as full_name
  from members
  limit 5

memid,zipcode,surname,firstname,temp_value,full_name
0,0,GUEST,GUEST,0,GUEST GUEST
1,4321,Smith,Darren,4322,Darren Smith
2,4321,Smith,Tracy,4323,Tracy Smith
3,23423,Rownam,Tim,23426,Tim Rownam
4,234,Joplette,Janice,238,Janice Joplette


###### join bookings and facilities

In [0]:
%sql
select bookings.facid as b_facid,
       bookings.memid,
       facilities.facid as f_facid,
       facilities.name

  from bookings
  left join facilities
  on bookings.facid = facilities.facid
  where name like '%Tennis Court%'

b_facid,memid,f_facid,name
0,2,0,Tennis Court 1
0,2,0,Tennis Court 1
1,0,1,Tennis Court 2
0,0,0,Tennis Court 1
0,0,0,Tennis Court 1
0,2,0,Tennis Court 1
0,2,0,Tennis Court 1
0,0,0,Tennis Court 1
0,2,0,Tennis Court 1
1,3,1,Tennis Court 2


###### join bookings, facilities, and members

In [0]:
%sql
select bookings.facid as b_facid,
       bookings.memid as b_memid,
       facilities.facid as f_facid,
       facilities.name,
       members.memid as m_memid,
       members.surname,
       members.firstname,
       concat_ws(' ', firstname, surname) as fullname

  from bookings
  left join facilities
  on bookings.facid = facilities.facid
  
  left join members
  on bookings.memid = members.memid
  
  where name like '%Tennis Court%'
  
  

b_facid,b_memid,f_facid,name,m_memid,surname,firstname,fullname
0,2,0,Tennis Court 1,2,Smith,Tracy,Tracy Smith
0,2,0,Tennis Court 1,2,Smith,Tracy,Tracy Smith
1,0,1,Tennis Court 2,0,GUEST,GUEST,GUEST GUEST
0,0,0,Tennis Court 1,0,GUEST,GUEST,GUEST GUEST
0,0,0,Tennis Court 1,0,GUEST,GUEST,GUEST GUEST
0,2,0,Tennis Court 1,2,Smith,Tracy,Tracy Smith
0,2,0,Tennis Court 1,2,Smith,Tracy,Tracy Smith
0,0,0,Tennis Court 1,0,GUEST,GUEST,GUEST GUEST
0,2,0,Tennis Court 1,2,Smith,Tracy,Tracy Smith
1,3,1,Tennis Court 2,3,Rownam,Tim,Tim Rownam


###### try to eliminate duplicates (this output is ugly)

In [0]:
%sql
select distinct(facilities.name, concat_ws(' ', firstname, surname)),
       facilities.name,
       concat_ws(' ', firstname, surname) as full_name
       

  from bookings
  left join facilities
  on bookings.facid = facilities.facid
  
  left join members
  on bookings.memid = members.memid
  
  where facilities.name like '%Tennis Court%'
  
  

"named_struct(name, name, col2, concat_ws( , firstname, surname))",name,full_name
"List(Tennis Court 1, Janice Joplette)",Tennis Court 1,Janice Joplette
"List(Tennis Court 1, Jack Smith)",Tennis Court 1,Jack Smith
"List(Tennis Court 1, Matthew Genting)",Tennis Court 1,Matthew Genting
"List(Tennis Court 2, Tracy Smith)",Tennis Court 2,Tracy Smith
"List(Tennis Court 2, Nancy Dare)",Tennis Court 2,Nancy Dare
"List(Tennis Court 1, John Hunt)",Tennis Court 1,John Hunt
"List(Tennis Court 1, Charles Owen)",Tennis Court 1,Charles Owen
"List(Tennis Court 1, Anne Baker)",Tennis Court 1,Anne Baker
"List(Tennis Court 2, Florence Bader)",Tennis Court 2,Florence Bader
"List(Tennis Court 1, Timothy Baker)",Tennis Court 1,Timothy Baker


###### Q7: ANSWER

In [0]:
%sql
select 
       facilities.name,
       concat_ws(' ', firstname, surname) as full_name,
       count(facilities.name) as num_res

  from bookings
  left join facilities
  on bookings.facid = facilities.facid
  
  left join members
  on bookings.memid = members.memid
  
  where name like '%Tennis Court%'
  
  group by facilities.name, full_name
  
  order by full_name
  
  

name,full_name,num_res
Tennis Court 1,Anne Baker,6
Tennis Court 2,Anne Baker,35
Tennis Court 2,Burton Tracy,3
Tennis Court 1,Burton Tracy,31
Tennis Court 2,Charles Owen,41
Tennis Court 1,Charles Owen,17
Tennis Court 2,Darren Smith,19
Tennis Court 2,David Farrell,1
Tennis Court 1,David Farrell,6
Tennis Court 1,David Jones,25


#### Q8: How can you produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than $30? 

- Remember that guests have different costs to members (the listed costs are per half-hour 'slot')
- The guest user's ID is always 0. 

#### Include in your output the name of the facility, the name of the member formatted as a single column, and the cost.

- Order by descending cost, and do not use any subqueries.

###### I did Q9 before this one so look at it to see steps that got me to the next cell
create column with daily cost   
create column indicating if day is the one of interest

In [0]:
%sql
select bookings.facid as b_facid,
       bookings.memid as b_memid,
       bookings.starttime,
       bookings.slots,
       facilities.name as f_name,
       facilities.membercost,
       facilities.guestcost,
       members.memid as m_memid,
       members.surname,
       members.firstname,
       
       case when members.memid = 0 then bookings.slots * facilities.guestcost
            else bookings.slots * facilities.membercost
            end
            as the_cost,
      
       case when bookings.starttime like '%2012-09-14%' then 1
            else 0
            end
            as select_day
   
  from bookings
  left join facilities
  on bookings.facid = facilities.facid
  
  left join members
  on bookings.memid = members.memid
 


b_facid,b_memid,starttime,slots,f_name,membercost,guestcost,m_memid,surname,firstname,the_cost,select_day
3,1,2012-07-03 11:00:00,2,Table Tennis,0.0,5.0,1,Smith,Darren,0.0,0
4,1,2012-07-03 08:00:00,2,Massage Room 1,9.9,80.0,1,Smith,Darren,19.8,0
6,0,2012-07-03 18:00:00,2,Squash Court,3.5,17.5,0,GUEST,GUEST,35.0,0
7,1,2012-07-03 19:00:00,2,Snooker Table,0.0,5.0,1,Smith,Darren,0.0,0
8,1,2012-07-03 10:00:00,1,Pool Table,0.0,5.0,1,Smith,Darren,0.0,0
8,1,2012-07-03 15:00:00,1,Pool Table,0.0,5.0,1,Smith,Darren,0.0,0
0,2,2012-07-04 09:00:00,3,Tennis Court 1,5.0,25.0,2,Smith,Tracy,15.0,0
0,2,2012-07-04 15:00:00,3,Tennis Court 1,5.0,25.0,2,Smith,Tracy,15.0,0
4,3,2012-07-04 13:30:00,2,Massage Room 1,9.9,80.0,3,Rownam,Tim,19.8,0
4,0,2012-07-04 15:00:00,2,Massage Room 1,9.9,80.0,0,GUEST,GUEST,160.0,0


###### almost answer, using member id

In [0]:
%sql
select case when bookings.starttime like '%2012-09-14%' then 1
            else 0
            end
            as select_day,
  
      members.memid,
      facilities.name,
      
      sum( case when members.memid = 0 then bookings.slots * facilities.guestcost
            else bookings.slots * facilities.membercost
            end)
      as total_day_cost


  from bookings
  left join facilities
  on bookings.facid = facilities.facid
  
  left join members
  on bookings.memid = members.memid
  
  group by (1), members.memid, facilities.name
  
  having total_day_cost > 30 and select_day = 1
  
  order by total_day_cost desc
  


select_day,memid,name,total_day_cost
1,0,Massage Room 1,480
1,0,Massage Room 2,320
1,0,Tennis Court 2,225
1,0,Tennis Court 1,150
1,0,Squash Court,140
1,13,Massage Room 1,59.400000000000006


###### Q8: ANSWER

In [0]:
%sql
select case when bookings.starttime like '%2012-09-14%' then 1
            else 0
            end
            as select_day,
  
      concat_ws(' ', members.firstname, members.surname) as full_name,
      facilities.name,
      
      sum( case when members.memid = 0 then bookings.slots * facilities.guestcost
            else bookings.slots * facilities.membercost
            end)
      as total_day_cost


  from bookings
  left join facilities
  on bookings.facid = facilities.facid
  
  left join members
  on bookings.memid = members.memid
  
  group by (1), (2), facilities.name
  
  having total_day_cost > 30 and select_day = 1
  
  order by total_day_cost desc
  


select_day,full_name,name,total_day_cost
1,GUEST GUEST,Massage Room 1,480.0
1,GUEST GUEST,Massage Room 2,320.0
1,GUEST GUEST,Tennis Court 2,225.0
1,GUEST GUEST,Tennis Court 1,150.0
1,GUEST GUEST,Squash Court,140.0
1,Jemima Farrell,Massage Room 1,59.400000000000006


#### Q9: This time, produce the same result as in Q8, but using a subquery.

###### get all bookings on that day

In [0]:
%sql
select facid as b_facid,
       memid as b_memid,
       starttime,
       slots
  from bookings
  where starttime like '%2012-09-14%'

b_facid,b_memid,starttime,slots
0,6,2012-09-14 08:00:00,3
0,17,2012-09-14 10:00:00,3
0,5,2012-09-14 12:30:00,3
0,3,2012-09-14 14:00:00,3
0,0,2012-09-14 16:00:00,3
0,26,2012-09-14 17:30:00,3
0,0,2012-09-14 19:00:00,3
1,11,2012-09-14 08:00:00,6
1,8,2012-09-14 11:00:00,6
1,0,2012-09-14 14:00:00,3


###### do joins

In [0]:
%sql
select bookings.facid as b_facid,
       bookings.memid as b_memid,
       bookings.starttime,
       bookings.slots,
       facilities.name as f_name,
       facilities.membercost,
       facilities.guestcost,
       members.memid as m_memid,
       members.surname,
       members.firstname
  
  from bookings
  left join facilities
  on bookings.facid = facilities.facid
  
  left join members
  on bookings.memid = members.memid
  
  where starttime like '%2012-09-14%'

b_facid,b_memid,starttime,slots,f_name,membercost,guestcost,m_memid,surname,firstname
0,6,2012-09-14 08:00:00,3,Tennis Court 1,5.0,25.0,6,Tracy,Burton
0,17,2012-09-14 10:00:00,3,Tennis Court 1,5.0,25.0,17,Pinker,David
0,5,2012-09-14 12:30:00,3,Tennis Court 1,5.0,25.0,5,Butters,Gerald
0,3,2012-09-14 14:00:00,3,Tennis Court 1,5.0,25.0,3,Rownam,Tim
0,0,2012-09-14 16:00:00,3,Tennis Court 1,5.0,25.0,0,GUEST,GUEST
0,26,2012-09-14 17:30:00,3,Tennis Court 1,5.0,25.0,26,Jones,Douglas
0,0,2012-09-14 19:00:00,3,Tennis Court 1,5.0,25.0,0,GUEST,GUEST
1,11,2012-09-14 08:00:00,6,Tennis Court 2,5.0,25.0,11,Jones,David
1,8,2012-09-14 11:00:00,6,Tennis Court 2,5.0,25.0,8,Boothe,Tim
1,0,2012-09-14 14:00:00,3,Tennis Court 2,5.0,25.0,0,GUEST,GUEST


###### add column for calculated cost

In [0]:
%sql
select bookings.facid as b_facid,
       bookings.memid as b_memid,
       bookings.starttime,
       bookings.slots,
       facilities.name as f_name,
       facilities.membercost,
       facilities.guestcost,
       members.memid as m_memid,
       members.surname,
       members.firstname,
       
       case when members.memid = 0 then bookings.slots * facilities.guestcost
            else bookings.slots * facilities.membercost
            end
            as the_cost
  
  from bookings
  left join facilities
  on bookings.facid = facilities.facid
  
  left join members
  on bookings.memid = members.memid
  
  where starttime like '%2012-09-14%'

b_facid,b_memid,starttime,slots,f_name,membercost,guestcost,m_memid,surname,firstname,the_cost
0,6,2012-09-14 08:00:00,3,Tennis Court 1,5.0,25.0,6,Tracy,Burton,15.0
0,17,2012-09-14 10:00:00,3,Tennis Court 1,5.0,25.0,17,Pinker,David,15.0
0,5,2012-09-14 12:30:00,3,Tennis Court 1,5.0,25.0,5,Butters,Gerald,15.0
0,3,2012-09-14 14:00:00,3,Tennis Court 1,5.0,25.0,3,Rownam,Tim,15.0
0,0,2012-09-14 16:00:00,3,Tennis Court 1,5.0,25.0,0,GUEST,GUEST,75.0
0,26,2012-09-14 17:30:00,3,Tennis Court 1,5.0,25.0,26,Jones,Douglas,15.0
0,0,2012-09-14 19:00:00,3,Tennis Court 1,5.0,25.0,0,GUEST,GUEST,75.0
1,11,2012-09-14 08:00:00,6,Tennis Court 2,5.0,25.0,11,Jones,David,30.0
1,8,2012-09-14 11:00:00,6,Tennis Court 2,5.0,25.0,8,Boothe,Tim,30.0
1,0,2012-09-14 14:00:00,3,Tennis Court 2,5.0,25.0,0,GUEST,GUEST,75.0


###### calculate total cost for day

In [0]:
%sql
select m_memid,
       f_name,
       sum(the_cost) as total_day_cost

from
(
select bookings.facid as b_facid,
       bookings.memid as b_memid,
       bookings.starttime,
       bookings.slots,
       facilities.name as f_name,
       facilities.membercost,
       facilities.guestcost,
       members.memid as m_memid,
       members.surname,
       members.firstname,
       
       case when members.memid = 0 then bookings.slots * facilities.guestcost
            else bookings.slots * facilities.membercost
            end
            as the_cost
  
  from bookings
  left join facilities
  on bookings.facid = facilities.facid
  
  left join members
  on bookings.memid = members.memid
  
  where starttime like '%2012-09-14%'
)

group by m_memid, f_name

order by m_memid, f_name
  


m_memid,f_name,total_day_cost
0,Massage Room 1,480.0
0,Massage Room 2,320.0
0,Squash Court,140.0
0,Tennis Court 1,150.0
0,Tennis Court 2,225.0
1,Badminton Court,0.0
2,Pool Table,0.0
3,Pool Table,0.0
3,Tennis Court 1,15.0
5,Badminton Court,0.0


###### almost answer, using member id

In [0]:
%sql
select m_memid,
       f_name,
       sum(the_cost) as total_day_cost

from
(
select bookings.facid as b_facid,
       bookings.memid as b_memid,
       bookings.starttime,
       bookings.slots,
       facilities.name as f_name,
       facilities.membercost,
       facilities.guestcost,
       members.memid as m_memid,
       members.surname,
       members.firstname,
       
       case when members.memid = 0 then bookings.slots * facilities.guestcost
            else bookings.slots * facilities.membercost
            end
            as the_cost
  
  from bookings
  left join facilities
  on bookings.facid = facilities.facid
  
  left join members
  on bookings.memid = members.memid
  
  where starttime like '%2012-09-14%'
)

group by m_memid, f_name

having total_day_cost > 30

order by total_day_cost desc
  


m_memid,f_name,total_day_cost
0,Massage Room 1,480
0,Massage Room 2,320
0,Tennis Court 2,225
0,Tennis Court 1,150
0,Squash Court,140
13,Massage Room 1,59.400000000000006


###### Q9: ANSWER (almost, could not figure out combining strings for member name)

In [0]:
%sql
select full_name,
       f_name,
       sum(the_cost) as total_day_cost

from
(
select bookings.facid as b_facid,
       bookings.memid as b_memid,
       bookings.starttime,
       bookings.slots,
       facilities.name as f_name,
       facilities.membercost,
       facilities.guestcost,
       members.memid as m_memid,
       members.surname,
       members.firstname,
       
       concat_ws(' ', members.firstname, members.surname) as full_name,
       
       case when members.memid = 0 then bookings.slots * facilities.guestcost
            else bookings.slots * facilities.membercost
            end
            as the_cost
  
  from bookings
  left join facilities
  on bookings.facid = facilities.facid
  
  left join members
  on bookings.memid = members.memid
  
  where starttime like '%2012-09-14%'
)

group by full_name, f_name

having total_day_cost > 30

order by total_day_cost desc
  


full_name,f_name,total_day_cost
GUEST GUEST,Massage Room 1,480.0
GUEST GUEST,Massage Room 2,320.0
GUEST GUEST,Tennis Court 2,225.0
GUEST GUEST,Tennis Court 1,150.0
GUEST GUEST,Squash Court,140.0
Jemima Farrell,Massage Room 1,59.400000000000006


#### Q10: Produce a list of facilities with a total revenue less than 1000.
- The output should have facility name and total revenue, sorted by revenue. 
- Remember that there's a different cost for guests and members!

###### Q10: ANSWER

In [0]:
%sql
select facilities.name,
      
      sum( case when members.memid = 0 then bookings.slots * facilities.guestcost
            else bookings.slots * facilities.membercost
            end)
      as total_revenue

  from bookings
  left join facilities
  on bookings.facid = facilities.facid
  
  left join members
  on bookings.memid = members.memid
  
  group by facilities.name
  
  having total_revenue < 1000
  
  order by total_revenue desc



name,total_revenue
Pool Table,270.0
Snooker Table,240.0
Table Tennis,180.0
